In [2]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv


In [3]:
!git clone https://github.com/yhlee1627/deeplearning.git
dataframe = read_csv("deeplearning/corona_daily.csv", usecols=[3], engine="python", skipfooter=3)
print(dataframe)
dataset = dataframe.values
dataset = dataset.astype('float32')


     Confirmed
0           24
1           24
2           27
3           27
4           28
..         ...
107      11190
108      11206
109      11225
110      11265
111      11344

[112 rows x 1 columns]


fatal: destination path 'deeplearning' already exists and is not an empty directory.


In [4]:
scaler = MinMaxScaler(feature_range=(0, 1))
Dataset = scaler.fit_transform(dataset)
#print(Dataset)
train_data, test_data = train_test_split(Dataset, test_size=0.2, shuffle=False)
print(len(train_data), len(test_data))

89 23


In [5]:
def create_dataset(dataset, look_back):
    x_data = []
    y_data = []
    for i in range(len(dataset) - look_back):
        data = dataset[i:(i + look_back),0]
        x_data.append(data)
        y_data.append(dataset[i + look_back, 0])
    return np.array(x_data), np.array(y_data)

In [6]:
look_back = 3
x_train, y_train = create_dataset(train_data, look_back)
x_test, y_test = create_dataset(test_data, look_back)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(86, 3) (86,)
(20, 3) (20,)


In [4]:
import gradio as gr

def greet(name):
    return "return " + name + "!"

demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch(debug=True, share=True)

TypeError: Interface.__init__() missing 2 required positional arguments: 'inputs' and 'outputs'